C:\Users\admin>tesseract-4.0.0 C:\BluePrism\Demo\InvoiceDemo\Invoice_1.jpg test   
Tesseract Open Source OCR Engine v4.0.0 with Leptonica  
Detected 29 diacritics  
  
Normalement un fichier test.txt est créé dans le répertoire courant

In [35]:
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
import json
import cv2
import numpy as np

In [36]:
#pytesseract.pytesseract.tesseract_cmd = r'tesseract-4.0.0.exe'
#fichier = r'/home/benoit/git/python_tutos/tesseract/tessFactures/Facture_1.jpg'
fichier = r'Facture_3.jpg'
image = Image.open(fichier)
output = {}

In [37]:
# test pour intégration flask
# https://gist.github.com/kylehounslow/767fb72fde2ebdd010a0bf4242371594
# coté client
img = cv2.imread(r'Facture_3.jpg')
_, img_encoded = cv2.imencode('.jpg', img)

# coté serveur
nparr = np.frombuffer(img_encoded.tostring(), np.uint8)
imgs = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
resultat = pytesseract.image_to_string(imgs)

In [38]:
resultat = pytesseract.image_to_string(image)
print(resultat)

BLUEPRISM

9 rue du colisée
75008 PARIS

+33 1 54 47 89 89
www.blueprism.com/fr

Référence: ZR980990
Date: 02/10/2018
Client: 456456456

Intitulé: Rénovation de la tour eiffel

FACTURE

Ma société
21 impasse de jumeaux
88000 EPINAL

Quantité Désignation Prix unitaire HT — Prix total HT
10 Crayons de papier HB 1,2 12
25 Papier canson 12g 3 50
4 Papier canson 20g 3,3 44
Total HT 66,40 €
TVA (20%) 12,80 €
Total TTC (en euros) 79,20 €

En votre aimable réglement,
Cordialement,

Conditions de paiement : paiement a réception de facture, a 30 jours...

Aucun escompte consenti pour reglement anticipé

Tout incident de paiement est passible d'intérét de retard. Le montant des pénalités résulte de l'application
aux sommes restant dues d'un taux d'intérét legal en vigueur au moment de I'incident.
Indemnité forfaitaire pour frais de recouvrement due au créancier en cas de retard de paiement : 40€

N° Siret 210.896.764 00015 RCS Montpellier


## Récupère l'adresse

In [4]:
def RemoveEmptyLines(entree):
    tab = entree.strip()
    tableausansvide = [ x for x in tab.splitlines() if x!='' ]
    res = ''
    for i in range(0, len(tableausansvide)):
        res = res + tableausansvide[i] + '\n'
    return res

def getTextBetween(mainString, startWord, endWord):
    start = mainString.find(startWord) + len(startWord)
    end = mainString.find(endWord)
    return RemoveEmptyLines(mainString[start:end])

output["Adresse"] = getTextBetween(resultat, 'www.blueprism.com/fr', 'Référence').strip()
print (output)

{'Adresse': 'Compagnie d’eléectricité\n3 avenue Foch\n75001 PARIS'}


## Récupère la référence

In [5]:
output["Reference"] = getTextBetween(resultat, 'Référence: ', 'Date: ').strip()
print (output)

{'Adresse': 'Compagnie d’eléectricité\n3 avenue Foch\n75001 PARIS', 'Reference': 'ZR8978989'}


## Récupère la date de facture

In [6]:
output["DateFacture"] = getTextBetween(resultat, 'Date: ', 'Client: ').strip()
print (output)

{'Adresse': 'Compagnie d’eléectricité\n3 avenue Foch\n75001 PARIS', 'Reference': 'ZR8978989', 'DateFacture': '06/12/2020'}


## Récupération du code client

In [7]:
output["CodeClient"] = getTextBetween(resultat, 'Client: ', 'Intitulé: ').strip()
print (output)

{'Adresse': 'Compagnie d’eléectricité\n3 avenue Foch\n75001 PARIS', 'Reference': 'ZR8978989', 'DateFacture': '06/12/2020', 'CodeClient': '897695146'}


## Récupération des details PO

In [8]:
pos = getTextBetween(resultat, 'Prix total HT', 'Total HT ')
tabPOs = pos.splitlines()
print ('Nombre de PO: ' + str(len(tabPOs)))
output["NbPo"] = len(tabPOs)
print (output)

Nombre de PO: 2
{'Adresse': 'Compagnie d’eléectricité\n3 avenue Foch\n75001 PARIS', 'Reference': 'ZR8978989', 'DateFacture': '06/12/2020', 'CodeClient': '897695146', 'NbPo': 2}


In [9]:
def getPosElement(po):
    element = {}
    element['quantite'] = po[0:po.find (' ')].strip()
    po = po[po.find (' '):len(po)]
    element['prixtotht'] = po[po.rfind (' '):len(po)].strip()
    po = po[0:po.rfind (' ')]
    element['prixunitht'] = po[po.rfind (' '):len(po)].strip()
    po = po[0:po.rfind (' ')]
    element['decription'] = po.strip()
    return element

print ('Ligne 1: ' + str(getPosElement(tabPOs[0])))

Ligne 1: {'quantite': '45', 'prixtotht': '541', 'prixunitht': '36', 'decription': 'Briquette'}


### Ajoute les lignes de factures

In [10]:
pos = []
for i in range(0, len(tabPOs)):
    pos.append(getPosElement(tabPOs[i]))
output['po'] = pos
print (output)

{'Adresse': 'Compagnie d’eléectricité\n3 avenue Foch\n75001 PARIS', 'Reference': 'ZR8978989', 'DateFacture': '06/12/2020', 'CodeClient': '897695146', 'NbPo': 2, 'po': [{'quantite': '45', 'prixtotht': '541', 'prixunitht': '36', 'decription': 'Briquette'}, {'quantite': '25', 'prixtotht': '410', 'prixunitht': '120', 'decription': 'Planches de chantier'}]}


## Total HT

In [11]:
output["totalht"] = getTextBetween(resultat, 'Total HT ', 'TVA (20%) ').strip()
print (output)

{'Adresse': 'Compagnie d’eléectricité\n3 avenue Foch\n75001 PARIS', 'Reference': 'ZR8978989', 'DateFacture': '06/12/2020', 'CodeClient': '897695146', 'NbPo': 2, 'po': [{'quantite': '45', 'prixtotht': '541', 'prixunitht': '36', 'decription': 'Briquette'}, {'quantite': '25', 'prixtotht': '410', 'prixunitht': '120', 'decription': 'Planches de chantier'}], 'totalht': '951,00 €'}


## TVA

In [12]:
output["tva"] = getTextBetween(resultat, 'TVA (20%) ', 'Total TTC (en euros) ').strip()
print (output)

{'Adresse': 'Compagnie d’eléectricité\n3 avenue Foch\n75001 PARIS', 'Reference': 'ZR8978989', 'DateFacture': '06/12/2020', 'CodeClient': '897695146', 'NbPo': 2, 'po': [{'quantite': '45', 'prixtotht': '541', 'prixunitht': '36', 'decription': 'Briquette'}, {'quantite': '25', 'prixtotht': '410', 'prixunitht': '120', 'decription': 'Planches de chantier'}], 'totalht': '951,00 €', 'tva': '190,20 €'}


## Total

In [14]:
output["total"] = getTextBetween(resultat, 'Total TTC (en euros) ', 'En votre aimable réglement,').strip()
print (output)

{'Adresse': 'Compagnie d’eléectricité\n3 avenue Foch\n75001 PARIS', 'Reference': 'ZR8978989', 'DateFacture': '06/12/2020', 'CodeClient': '897695146', 'NbPo': 2, 'po': [{'quantite': '45', 'prixtotht': '541', 'prixunitht': '36', 'decription': 'Briquette'}, {'quantite': '25', 'prixtotht': '410', 'prixunitht': '120', 'decription': 'Planches de chantier'}], 'totalht': '951,00 €', 'tva': '190,20 €', 'total': '1041,20€'}
